# Лабараторная работа 1
Вариант 11

#### **Цель работы:**
Научиться реализовывать один из алгоритмов глубокого обучения.

#### **Задание:**
1. Скачайте датасет [CarDatasets](https://drive.google.com/drive/folders/1pkudEBabqbXMxRTgfGQs3e0VqfTjtqWU)
2. Реализуйте ResNet-101 с оптимизатором [Adabound](https://arxiv.org/abs/1902.09843v1) с использованием
Numpy и с **Torch**/Tensorflow/Jax
3. Оцените качество модели на тесте и сравните быстродействие
реализованных вариантов.
4. Запустить обучение на классическом Adam и сравнить сходимость
результатов с вариантом задания.
5. Сделайте отчёт в виде readme на GitHub, там же должен быть выложен
исходный код.

#### **Отчёт должен содержать следующие пункты:**
1. Теоретическая база
2. Описание разработанной системы (алгоритмы, принципы работы,
архитектура)
3. Результаты работы и тестирования системы (скриншоты, изображения,
графики, закономерности)
4. Выводы по работе
5. Использованные источники

План:
1. Получить Dataloader, возвращающий обрезанное в соответствии с annotations изображение, убедиться, что тензор корректно конвертируется в numpy
2. Загрузить и запустить готовый resnet101. Хотя, скорее всего, это не пригодится. Запустить на нём обучение
3. Имплементировать на torch
4. Имплементирвать на numpy


In [8]:
import scipy.io
import numpy as np
import torch
import torchvision

In [2]:
model = torchvision.models.resnet101()

In [3]:
lr_loc = r'C:\Users\SystemPoint\Downloads\LR1-1'

In [4]:
# get a list of class names
meta = scipy.io.loadmat(lr_loc+'\cars_meta.mat', squeeze_me = True)
class_names = meta['class_names']
class_names[:4]

array(['AM General Hummer SUV 2000', 'Acura RL Sedan 2012',
       'Acura TL Sedan 2012', 'Acura TL Type-S 2008'], dtype=object)

In [5]:
train_annos = scipy.io.loadmat(lr_loc+'\cars_train_annos.mat', squeeze_me = True)
train_annos['annotations']

array([(39, 116, 569, 375, 14, '00001.jpg'),
       (36, 116, 868, 587, 3, '00002.jpg'),
       (85, 109, 601, 381, 91, '00003.jpg'), ...,
       (26, 246, 660, 449, 163, '08142.jpg'),
       (78, 526, 1489, 908, 112, '08143.jpg'),
       (20, 240, 862, 677, 17, '08144.jpg')],
      dtype=[('bbox_x1', 'O'), ('bbox_y1', 'O'), ('bbox_x2', 'O'), ('bbox_y2', 'O'), ('class', 'O'), ('fname', 'O')])

In [6]:
test_annos = scipy.io.loadmat(lr_loc+'\cars_test_annos.mat', squeeze_me = True)
train_annos['annotations']

array([(39, 116, 569, 375, 14, '00001.jpg'),
       (36, 116, 868, 587, 3, '00002.jpg'),
       (85, 109, 601, 381, 91, '00003.jpg'), ...,
       (26, 246, 660, 449, 163, '08142.jpg'),
       (78, 526, 1489, 908, 112, '08143.jpg'),
       (20, 240, 862, 677, 17, '08144.jpg')],
      dtype=[('bbox_x1', 'O'), ('bbox_y1', 'O'), ('bbox_x2', 'O'), ('bbox_y2', 'O'), ('class', 'O'), ('fname', 'O')])